# Word2Vec 학습
## Tagging 된 CSV 가져오기
- 공통 변수 선언

In [4]:
import os 
import pandas as pd

PROJ_HOME = '\\'.join(os.getcwd().split('\\')[0:-1])
MARK_LST_CSV = f'{PROJ_HOME}\\crawled_files\\market_master.csv'
OKT_TAG_DIR = f'{PROJ_HOME}\\okt_tag_files'

In [1]:
stop_words = ['이','날','전','에','인데','엇','가','도','거','이랑','랑','은','너','요','면','할','것','의','있습니다','는','식']

In [5]:
all_tag_review_lst = []
for okt_tag_file in os.listdir(OKT_TAG_DIR):
    file = f'{OKT_TAG_DIR}\\{okt_tag_file}'
    review_df = pd.read_csv(file, encoding='utf-8')
    review_df = review_df.dropna(axis=0)
    for idx, row in review_df.iterrows():
        tagged_text = row['tagged_text']
        tagged_text_split_lst = tagged_text.split(' ')
        tagged_text_split_lst = [word for word in tagged_text_split_lst if word not in stop_words] 
        all_tag_review_lst.append(tagged_text_split_lst)

print(all_tag_review_lst[:3])

[['물회', '회덮밥', '맛집', '오후', '시', '쯤', '도착', '해서', '이른', '저녁', '먹었어요', '회도', '싱싱하고', '야채', '많아', '정말', '배부르게', '먹었네요', '함께', '나오는', '생선', '구이', '짱', '맛', '입니다'], ['부모님', '가족', '들', '과', '다', '같이', '오고', '싶은', '곳', '예요', '정말', '만족', '스럽게', '잘', '먹었습니다', '다음', '또', '올게', '용'], ['셋트이름', '생각', '안나', '지만', '사진', '으로', '표현', '해드려요', '회', '나', '해산물', '신선하고요', '찜', '맛있어요', '전체', '적', '으로', '음식', '맛있어요', '다만', '초밥', '밥', '건조해져서', '비추', '입니다', '층', '으로', '되어있어서', '넓고', '쾌적합니', '다', '이집', '추천', '해요']]


## Word2vec 학습시키기 
- 파라미터는 아래 참고 
    * vector_size: 차원의 크기, 100차원 120차원, 200차원 등 
    * window: 문장 내 중심 단어를 포함하여 양 옆으로 인접할 단어의 개수 
    * min_count: 단어의 최소 빈도수 제한 (이보다 작은 빈도의 단어는 미학습함)
    * workers: 학습을 위한 프로세스 개수 
    * sg: 0은 CBOW, 1은 SKIP-GRAM

In [6]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=all_tag_review_lst , vector_size = 100 , window = 5, min_count=3 , workers = 4 , sg = 0)

In [30]:
# 다른 값으로 바꿔가며 확인해보세요.
similar_rslt = model.wv.most_similar(positive=['짬뽕'], negative=['커피'])          # positive와는 유사하지만 negative와는 먼
print(similar_rslt)

[('붉은', 0.41380780935287476), ('b', 0.30172011256217957), ('a', 0.29738718271255493), ('순두부', 0.27546262741088867), ('물회', 0.23982565104961395), ('세트', 0.22132812440395355), ('회', 0.19141291081905365), ('편이었어요', 0.17840933799743652), ('간간', 0.17744366824626923), ('퀄', 0.15825864672660828)]


## 모델 저장, 모델 로드

In [14]:
model.wv.save_word2vec_format('./myw2vModel') #모델저장


In [15]:
# 모델 로드
#model.wv.load_word2vec_format('./myw2vModel')


## tsv 파일 생성
- projector.tensorflow.org 에 업로드용 tsv 파일 생성
- 두 개의 tsv 파일을 모두 projector.tensorflow.org 에 업로드하여 결과 확인 

In [7]:
TSV_DIR = f'{PROJ_HOME}\\tsv_files'
if not os.path.exists(TSV_DIR):
    os.mkdir(TSV_DIR)
    

with open(TSV_DIR + '\\tensors.tsv', 'w', encoding='utf-8') as tensors:
    with open(TSV_DIR + '\\metadata.tsv', 'w', encoding='utf-8') as metadata:
        for word in model.wv.index_to_key :
            metadata.write(word + '\n')
            vector_row = '\t'.join(map(str, model.wv[word]))
            tensors.write(vector_row + '\n')